In [1]:
import chess.pgn
import pandas as pd

In [1]:
def pgn_to_csv(player):
    file2 = open('pgns/' +player + '.pgn',encoding="utf-8",errors = "ignore")
    white = []
    black = []
    white_elo = []
    black_elo = []
    date = []
    event = []
    result = []
    wdiff = []
    bdiff = []
    rounds = []
    opening = []
    moves = []
    #TODO - change this to a more standard while loop
    game = chess.pgn.read_game(file2)
    while isinstance(game,chess.pgn.Game):
        if "White" in game.headers:
            white.append(game.headers["White"])
        else:
            white.append('*')
        if "Round" in game.headers:
            rounds.append(game.headers["Round"])
        else:
            white.append('*')
        if "Black" in game.headers:
            black.append(game.headers["Black"])
        else:
            black.append('*')
        if "WhiteElo" in game.headers:
            white_elo.append(game.headers["WhiteElo"])
        else:
            white_elo.append('*')
        if "BlackElo" in game.headers:
            black_elo.append(game.headers["BlackElo"])
        else:
            black_elo.append('*')
        if "Date" in game.headers:
            date.append(game.headers["Date"])
        else:
            date.append('*')
        if "EventType" in game.headers:
            event.append(game.headers["EventType"])
        else:
            event.append('*')
        if "ECO" in game.headers:
            opening.append(game.headers["ECO"])
        else:
            opening.append('*')
        if "Result" in game.headers:
            result.append(game.headers["Result"])
        else:
            result.append('*')
        game = chess.pgn.read_game(file2)

        data = {"White": white, "Black": black, "WhiteELO": white_elo, "BlackELO": black_elo, "Date": date,"Event": event, "Result": result, "Opening": opening, "Rounds": rounds}
        
        df = pd.DataFrame(data)
        df.to_csv('csvs/' + player +'.csv')

In [8]:
players = ['andreikin, dmitry', 'anand, viswanathan', 'wang, hao', 'grischuk, alexander', 'karjakin, sergey','duda, jan-krzysztof', 'radjabov, teimour', 'dominguez perez, leinier','nakamura, hikaru', 'vachier-lagrave, maxime','aronian, levon','mamedyarov, shakhriyar', 'so, wesley','ding, liren', 'rapport, richard', 'nepomniachtchi, ian', 'giri, anish', 'firouzja, alireza', 'caruana, fabiano','carlsen, magnus']
for player in players:
    pgn_to_csv(player)